## import library

In [ ]:
import os
import numpy as np
import tensorflow as tf
from data_loader import *
from data_utils import *
from model import *
from const import *

## prepare data

In [ ]:
train_data, test_data = getBagImage()

In [ ]:
def one_hot(index, num_classes):
    tmp = np.zeros(num_classes, dtype=np.float32)
    tmp[index] = 1.0
    return tmp

# main evaluation code

## input tensor + model network

In [ ]:
sess = tf.InteractiveSession()

x, y, mask = input_tensor()

y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_material_conv, y_handle_conv, y_decoration_conv, is_training, keep_prob = multi_label_net(x)

## loss + mask

In [ ]:
color_loss, shape_loss, opening_loss, strap_loss, pattern_loss, material_loss, handle_loss, decoration_loss, l2_loss, total_loss = selective_loss(y_color_conv, y_shape_conv, y_opening_conv, y_strap_conv, y_pattern_conv, y_material_conv, y_handle_conv, y_decoration_conv,y, mask)

train_step = train_op(total_loss, global_step)

color_mask = tf.get_collection('color_mask')[0]
shape_mask = tf.get_collection('shape_mask')[0]
opening_mask = tf.get_collection('opening_mask')[0]
strap_mask = tf.get_collection('strap_mask')[0]
pattern_mask = tf.get_collection('pattern_mask')[0]
material_mask = tf.get_collection('material_mask')[0]
handle_mask = tf.get_collection('handle_mask')[0]
decoration_mask = tf.get_collection('decoration_mask')[0]

y_color = tf.get_collection('y_color')[0]
y_shape = tf.get_collection('y_shape')[0]
y_opening = tf.get_collection('y_opening')[0]
y_strap = tf.get_collection('y_strap')[0]
y_pattern = tf.get_collection('y_pattern')[0]
y_material = tf.get_collection('y_material')[0]
y_handle = tf.get_collection('y_handle')[0]
y_decoration = tf.get_collection('y_decoration')[0]

color_correct_prediction = tf.equal(tf.argmax(y_color_conv, 1), tf.argmax(y_color, 1))
shape_correct_prediction = tf.equal(tf.argmax(y_shape_conv, 1), tf.argmax(y_shape, 1))
opening_correct_prediction = tf.equal(tf.argmax(y_opening_conv, 1), tf.argmax(y_opening, 1))
strap_correct_prediction = tf.equal(tf.argmax(y_strap_conv, 1), tf.argmax(y_strap, 1))
pattern_correct_prediction = tf.equal(tf.argmax(y_pattern_conv, 1), tf.argmax(y_pattern, 1))
material_correct_prediction = tf.equal(tf.argmax(y_material_conv, 1), tf.argmax(y_material, 1))
handle_correct_prediction = tf.equal(tf.argmax(y_handle_conv, 1), tf.argmax(y_handle, 1))
decoration_correct_prediction = tf.equal(tf.argmax(y_decoration_conv, 1), tf.argmax(y_decoration, 1))

color_true_pred = tf.reduce_sum(tf.cast(color_correct_prediction, dtype=tf.float32) * color_mask)
shape_true_pred = tf.reduce_sum(tf.cast(shape_correct_prediction, dtype=tf.float32) * shape_mask)
opening_true_pred = tf.reduce_sum(tf.cast(opening_correct_prediction, dtype=tf.float32) * opening_mask)
strap_true_pred = tf.reduce_sum(tf.cast(strap_correct_prediction, dtype=tf.float32) * strap_mask)
pattern_true_pred = tf.reduce_sum(tf.cast(pattern_correct_prediction, dtype=tf.float32) * pattern_mask)
material_true_pred = tf.reduce_sum(tf.cast(material_correct_prediction, dtype=tf.float32) * material_mask)
handle_true_pred = tf.reduce_sum(tf.cast(handle_correct_prediction, dtype=tf.float32) * handle_mask)
decoration_true_pred = tf.reduce_sum(tf.cast(decoration_correct_prediction, dtype=tf.float32) * decoration_mask)

real_test_data = []

# Mask : color -> 0 , shape -> 1, opening -> 2, strap -> 3, pattern -> 4, material -> 5, handle -> 6, decoration -> 7
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][0], 16), 0.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][1], 16), 1.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][2], 16), 2.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][3], 16), 3.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][4], 16), 4.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][5], 16), 5.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][6], 16), 6.0))
for i in range(len(train_data)):
    img = (train_data[i][0] - 128) / 255.0
    label = train_data[i][1]
    real_train_data.append((img, one_hot(label[i][7], 16), 7.0))

np.random.shuffle(test_data)

print('Restore model')
saver = tf.train.Saver()
saver.restore(sess, SAVE_FOLDER + 'model.ckpt')
print('OK')

In [ ]:
train_img = []
train_label = []
train_mask = []

for i in range(len(real_train_data)):
    train_img.append(real_train_data[i][0])
    train_label.append(real_train_data[i][1])
    train_mask.append(real_train_data[i][2])

number_batch = len(real_test_data) // BATCH_SIZE

color_nb_true_pred = 0
shape_nb_true_pred = 0
opening_nb_true_pred = 0
strap_nb_true_pred = 0
pattern_nb_true_pred = 0
material_nb_true_pred = 0
handle_nb_true_pred = 0
decoration_nb_true_pred = 0

color_nb_train = 0
shape_nb_train = 0
opening_nb_train = 0
strap_nb_train = 0
pattern_nb_train = 0
material_nb_train = 0
handle_nb_train = 0
decoration_nb_train = 0

for batch in range(number_batch):
    # print('Training on batch {0}/{1}'.format(str(batch + 1), str(number_batch)))
    top = batch * BATCH_SIZE
    bot = min((batch + 1) * BATCH_SIZE, len(real_train_data))
    batch_img = np.asarray(train_img[top:bot])
    batch_label = np.asarray(train_label[top:bot])
    batch_mask = np.asarray(train_mask[top:bot])
    
    batch_img = augmentation(batch_img, 64)

    for i in range(BATCH_SIZE):
        if batch_mask[i] == 0.0:
            color_nb_train += 1
        else:
            if batch_mask[i] == 1.0:
                shape_nb_train += 1
            else:
                if batch_mask[i] == 2.0:
                    opening_nb_train += 1
                else:
                    if batch_mask[i] == 3.0:
                        strap_nb_train += 1
                    else:
                        if batch_mask[i] == 4.0:
                            pattern_nb_train += 1
                        else:
                            if batch_mask[i] == 5.0:
                                material_nb_train += 1
                            else:
                                if batch_mask[i] == 6.0:
                                    handle_nb_train += 1
                                else:
                                    decoration_nb_train +=1

      

    color_nb_true_pred += sess.run(color_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False,keep_prob: 1})

    shape_nb_true_pred += sess.run(shape_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False, keep_prob: 1})

    opening_nb_true_pred += sess.run(opening_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: False, keep_prob: 1})

    strap_nb_true_pred += sess.run(strap_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                               is_training: False, keep_prob: 1})

    pattern_nb_true_pred += sess.run(pattern_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                   is_training: False, keep_prob: 1})

    material_nb_true_pred += sess.run(material_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob: 1})

    handle_nb_true_pred += sess.run(handle_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob:1})

    decoration_nb_true_pred += sess.run(decoration_true_pred, feed_dict={x: batch_img, y: batch_label, mask: batch_mask,
                                                                     is_training: False, keep_prob: 1})

    sess.run(update_op,feed_dict={x: batch_img, y_: batch_label, mask: batch_mask, is_training: False, keep_prob: 1})



color_train_accuracy = color_nb_true_pred * 1.0 / color_nb_train
shape_train_accuracy = shape_nb_true_pred * 1.0 / shape_nb_train
opening_train_accuracy = opening_nb_true_pred * 1.0 / opening_nb_train
strap_train_accuracy = strap_nb_true_pred * 1.0 / strap_nb_train
pattern_train_accuracy = pattern_nb_true_pred * 1.0 / pattern_nb_train
material_train_accuracy = material_nb_true_pred * 1.0 / material_nb_train
handle_train_accuracy = handle_nb_true_pred * 1.0 / handle_nb_train
decoration_train_accuracy = decoration_nb_true_pred * 1.0 / decoration_nb_train

print('\n')
print('color task train accuracy: ' + str(color_train_accuracy * 100))
print('shape task train accuracy: ' + str(shape_train_accuracy * 100))
print('opening task train accuracy: ' + str(opening_train_accuracy * 100))
print('strap task train accuracy: ' + str(strap_train_accuracy * 100))
print('pattern task train accuracy: ' + str(pattern_train_accuracy * 100))
print('material task train accuracy: ' + str(material_train_accuracy * 100))
print('handle task train accuracy: ' + str(handle_train_accuracy * 100))
print('decoration task train accuracy: ' + str(decoration_train_accuracy * 100))